In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install torch
!pip uninstall -y googletrans httpx httpcore
!pip install googletrans-py

Found existing installation: httpx 0.28.1
Uninstalling httpx-0.28.1:
  Successfully uninstalled httpx-0.28.1
Found existing installation: httpcore 1.0.9
Uninstalling httpcore-1.0.9:
  Successfully uninstalled httpcore-1.0.9
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.6 MB/s eta 0:00:00
  Created wheel for googletrans-py: filename=googletrans_py-4.0.0-py3-none-any.whl size=13115 sha256=1ca9eca87f77f8d16933760383f95d64bae0f8ff3e3cb2e0b640acc7ff35d05a
  Stored in directory: /root/.cache/pip/wheels/d7/77/86/56a6d2caf2420c4f8f062d014fdf011477f213718ef9aae7d2
Successfully built googletrans-py


In [2]:
from transformers import pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-ar")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [3]:
import pandas as pd

df = pd.read_excel("/content/merged_chatbot_dataset.xlsx")

df.head()


,intent,response,language,source_file,source_sheet,example
0,best_appliance_brand,"Top brands: El-Araby, B.TECH, Zanussi.",en,chatbot_dataset_english.xlsx,Sheet1,What is the best appliance brand?
1,best_appliance_brand,"Top brands: El-Araby, B.TECH, Zanussi.",en,chatbot_dataset_english.xlsx,Sheet1,Which appliance brand should I buy?
2,best_appliance_brand,"Top brands: El-Araby, B.TECH, Zanussi.",en,chatbot_dataset_english.xlsx,Sheet1,Best company for home appliances?
3,best_appliance_brand,"Top brands: El-Araby, B.TECH, Zanussi.",en,chatbot_dataset_english.xlsx,Sheet1,What brand is good for electronics?
4,best_appliance_brand,"Top brands: El-Araby, B.TECH, Zanussi.",en,chatbot_dataset_english.xlsx,Sheet1,Recommend a good appliance brand.


In [4]:
df["text"] = df["example"].astype(str)
df["label"] = df["intent"].astype("category").cat.codes

label2intent = dict(enumerate(df["intent"].astype("category").cat.categories))

In [5]:
from sklearn.model_selection import train_test_split

train_text, test_text, train_labels, test_labels = train_test_split(
    df["text"].tolist(),
    df["label"].tolist(),
    test_size=0.1,
    random_state=42
)


In [6]:
from transformers import AutoTokenizer

model_name = "distilbert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=128)



tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [7]:
from datasets import Dataset

train_ds = Dataset.from_dict({"text": train_text, "labels": train_labels})
test_ds = Dataset.from_dict({"text": test_text, "labels": test_labels})

train_ds = train_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)


Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(df["label"].unique())
)


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}


In [20]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./bert-intent",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics
)

In [12]:
import wandb
wandb.init(project="bert-intent-classification")
trainer.train()

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 4774b9ffed337a1378175ead44e2e1ee58f73144


wandb: WARNING Invalid choice
wandb: Enter your choice:

 4774b9ffed337a1378175ead44e2e1ee58f73144


wandb: WARNING Invalid choice
wandb: Enter your choice:

...

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,No log,1.958359
2,No log,1.530113
3,No log,1.153727
4,No log,0.955862
5,No log,0.884067


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=115, training_loss=1.44728539508322, metrics={'train_runtime': 230.3559, 'train_samples_per_second': 7.814, 'train_steps_per_second': 0.499, 'total_flos': 8942825088000.0, 'train_loss': 1.44728539508322, 'epoch': 5.0})

In [21]:
results = trainer.evaluate()
print("Evaluation results:", results)

Evaluation results: {'eval_loss': 0.8840667605400085, 'eval_model_preparation_time': 0.0059, 'eval_accuracy': 0.975, 'eval_runtime': 2.2129, 'eval_samples_per_second': 18.076, 'eval_steps_per_second': 2.259}


In [13]:
import torch

def predict_intent(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return label2intent[pred]


In [14]:
def bot_reply(message):
    intent = predict_intent(message)
    row = df[df["intent"] == intent].iloc[0]
    arabic_reply = None
    english_reply = None
    if row["language"] == "ar":
        arabic_reply = row["response"]
    elif row["language"] == "en":
        english_reply = row["response"]
    # If no Arabic reply, translate the English one using the pipeline
    if arabic_reply is None and english_reply is not None:
        try:
            arabic_reply = translator(english_reply)[0]['translation_text']
        except Exception as e:
            arabic_reply = "ترجمة غير متوفرة حالياً."  # Fallback
    return {
        "arabic": arabic_reply,
        "english": english_reply
    }

In [15]:
reply = bot_reply("  افضل غسالة")
print(reply)


{'arabic': 'أفضل علامات ماكينة الغسيل: زانوسي، شارب، سامسونغ.', 'english': 'Top washing machine brands: Zanussi, Sharp, Samsung.'}


In [16]:
reply = bot_reply("ايه افضل شركة اجهزة كهربائية؟")
print(reply)

{'arabic': 'العلامات التجارية العليا: العربية، B.TECH، زانوسي.', 'english': 'Top brands: El-Araby, B.TECH, Zanussi.'}


In [17]:
reply = bot_reply("Where can I buy ceramics?")
print(reply)

{'arabic': 'أفضل الأماكن: Ceramics Cliopatra, El-Gawhara, Real Ceramics.', 'english': 'Best places: Cleopatra Ceramics, El-Gawhara, Royal Ceramics.'}


In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.post("/chat")
def chat(message: str):
    return bot_reply(message)
